In [8]:
import getpass
import paramiko
import os

def get_password_for_machine(user, host):
    print(f"To set up simulation monitoring for {host}, I need to know your password")
    print("Don't worry, it will not be stored to disk.")
    passprompt = f"Please enter the password for {user}@{host}"
    return getpass.getpass(prompt=passprompt)

In [9]:
def generate_keypair(user, host):
    print(f"Generating a specific key for esm_viz to use for {host}")
    priv = paramiko.RSAKey.generate(2048)
    subpath = (".config", "esm_viz", "keys")
    keypath = os.path.join(os.environ.get("HOME"), *subpath, f"{user}_{host}")
    # Private Key:
    priv.write_private_key_file(keypath)
    # Public Key
    pub = paramiko.RSAKey(filename=keypath)
    with open(keypath+".pub", "w") as pub_key_file:
        pub_key_file.write("%s %s" % (pub.get_name(), pub.get_base64()))

In [21]:
def deploy_keypair(user, host):
    subpath = (".config", "esm_viz", "keys")
    priv_file = os.path.join(os.environ.get("HOME"), *subpath, f"{user}_{host}")+".priv_key"
    if not os.path.isfile(priv_file):
        generate_keypair(user, host)
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    # TODO: get user preference for the policy
    client.set_missing_host_key_policy(paramiko.WarningPolicy)
    remote_password = get_password_for_machine(user, host)
    client.connect(host, username=user, password=remote_password)
    sftp = client.open_sftp() 
    # If HOME isn't set....oh well...
    _, stdout, _ = client.exec_command("echo $HOME")
    remote_home = stdout.readlines()[0].strip()
    known_hosts_remote = os.path.join(remote_home, ".ssh/known_hosts")
    with sftp.open(known_hosts_remote, "a") as r_known_hosts:
        with open(priv_file+".pub", 'r') as esm_viz_pub_key:
            r_known_hosts.write(esm_viz_pub_key.readlines())
    print("Deleting your password from memory...")
    del remote_password
    

In [10]:
!mkdir -p ~/.config/esm_viz/keys

In [11]:
generate_keypair("a270077", "mistral.dkrz.de")

Generating a specifc key for esm_viz to use for mistral.dkrz.de


In [15]:
paramiko.RSAKey("/Users/pgierz/.config/esm_viz/keys/a270077_mistral.dkrz.de.priv_key")

AttributeError: 'str' object has no attribute 'rewind'

In [24]:
sftp = deploy_keypair("pgierz", "localhost")

To set up simulation monitoring for localhost, I need to know your password
Don't worry, it will not be stored to disk.


Please enter the password for pgierz@localhost ········


In [27]:
sftp.